In [ ]:
import pandas as pd
import ssl 
import os
import logging
from pathlib import Path
import time
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")
ssl._create_default_https_context = ssl._create_unverified_context


ssl._create_default_https_context = ssl._create_unverified_context

Table: 'career_avg_bowler'

In [ ]:
def career_avg_bowler(player_ids):

    career_avg_bowler_merged = pd.DataFrame()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for p_id in player_ids:
        url = "https://stats.espncricinfo.com/ci/engine/player/" + p_id + ".html?class=2;template=results;type=bowling"
        response = requests.get(url, headers=headers)

        dismissal_url = "https://stats.espncricinfo.com/ci/engine/player/" + p_id + ".html?class=2;template=results;type=bowling;view=dismissal_summary"
        dismissal_response = requests.get(dismissal_url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            tables = pd.read_html(str(soup))

            if len(tables) > 2 and "Span" in tables[2].columns:
                career_avg_bowler = tables[2]
                career_avg_bowler = career_avg_bowler[["Span","Inns","Overs","Mdns","Runs","Wkts","Ave","Econ","SR"]]
                career_avg_bowler.insert(0, "P_id", [p_id], True)
                
        if dismissal_response.status_code == 200:
            dismissal_soup = BeautifulSoup(dismissal_response.content, 'html.parser')
            dismissal_tables = pd.read_html(str(dismissal_soup))
                    
            if len(dismissal_tables) > 2 and "Grouping" in dismissal_tables[3].columns:
                dismissal = dismissal_tables[3]
                dismissal_types = ["caught", "bowled", "leg before wicket"]
                dismissal.set_index("Grouping", inplace=True)
                dismissal = dismissal.loc[dismissal_types, ["Dis"]]
                dismissal_dict = dismissal.to_dict()
                dismissal = pd.DataFrame.from_dict(dismissal_dict.values())
                        
                career_avg_bowler = pd.concat([career_avg_bowler, dismissal], axis=1)
                
                career_avg_bowler_merged = pd.concat([career_avg_bowler_merged, career_avg_bowler], ignore_index=True)
            time.sleep(0.1)
        else:
            print(f"Failed to retrieve data for player ID {p_id}, status code: {response.status_code}")

    filepath = Path('Csv/Career_avg_bowler.csv')
    career_avg_bowler_merged.to_csv(filepath, index_label="row_id")
    print(career_avg_bowler_merged)


Table: 'vs_country_bowler'

In [ ]:
def vs_country_bowler(player_ids):

    vs_country_bowler_merged = pd.DataFrame()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for p_id in player_ids:
        url = "https://stats.espncricinfo.com/ci/engine/player/" + p_id + ".html?class=2;template=results;type=bowling"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            tables = pd.read_html(str(soup))

            if len(tables) > 3:
                vs_country_bowler = tables[3]
                vs_country_bowler = vs_country_bowler[["Grouping","Inns","Overs","Mdns","Runs","Wkts","Ave","Econ","SR"]]
                vs_country_bowler.rename(columns={'Grouping': 'Country'}, inplace=True)
                vs_country_bowler.insert(0, "P_id", p_id, True)

                countries = ['v Pakistan', 'v Australia', 'v England', 'v New Zealand', 'v West Indies', 'v Sri Lanka', 'v Bangladesh', 'v South Africa']
                select_vs_country_bowler = vs_country_bowler.loc[vs_country_bowler['Country'].isin(countries)]
                
                vs_country_bowler_merged = pd.concat([vs_country_bowler_merged, select_vs_country_bowler], ignore_index=True)
            time.sleep(0.1)
        else:
            print(f"Failed to retrieve data for player ID {p_id}, status code: {response.status_code}")

    filepath = Path('Csv/Vs_country_bowler.csv')
    vs_country_bowler_merged.to_csv(filepath, index_label="row_id")
    print(vs_country_bowler_merged)

Table: 'home_vs_away_bowler'

In [ ]:
def home_vs_away_bowler(player_ids):

    home_vs_away_bowler_merged = pd.DataFrame()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for p_id in player_ids:
        url = "https://stats.espncricinfo.com/ci/engine/player/" + p_id + ".html?class=2;template=results;type=bowling"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            tables = pd.read_html(str(soup))

            if len(tables) > 3:
                home_vs_away_bowler = tables[3]
                home_vs_away_bowler = home_vs_away_bowler[["Grouping","Inns","Overs","Mdns","Runs","Wkts","Ave","Econ","SR"]]
                home_vs_away_bowler.rename(columns={'Grouping': 'Venue'}, inplace=True)
                home_vs_away_bowler.insert(0, "P_id", p_id, True)

                venues = ['home', 'away', 'neutral']
                select_home_vs_away_bowler = home_vs_away_bowler.loc[home_vs_away_bowler['Venue'].isin(venues)]
                
                home_vs_away_bowler_merged = pd.concat([home_vs_away_bowler_merged, select_home_vs_away_bowler], ignore_index=True)
            time.sleep(0.1)
        else:
            print(f"Failed to retrieve data for player ID {p_id}, status code: {response.status_code}")

    filepath = Path('Csv/Home_vs_away_bowler.csv')
    home_vs_away_bowler_merged.to_csv(filepath, index_label="row_id")
    print(home_vs_away_bowler_merged)

Table: 'yearly_stats_bowler'

In [ ]:
def yearly_stats_bowler(player_ids):

    yearly_stats_bowler_merged = pd.DataFrame()
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for p_id in player_ids:
        url = "https://stats.espncricinfo.com/ci/engine/player/" + p_id + ".html?class=2;template=results;type=bowling"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            tables = pd.read_html(str(soup))

            if len(tables) > 3:
                yearly_stats_bowler = tables[3]
                yearly_stats_bowler = yearly_stats_bowler[["Grouping","Inns","Overs","Mdns","Runs","Wkts","Ave","Econ","SR"]]
                yearly_stats_bowler.rename(columns={'Grouping': 'Year'}, inplace=True)
                yearly_stats_bowler.insert(0, "P_id", p_id, True)

                select_yearly_stats_bowler = yearly_stats_bowler.loc[yearly_stats_bowler['Year'].str.startswith('year ', na=False)]
                yearly_stats_bowler_merged = pd.concat([yearly_stats_bowler_merged, select_yearly_stats_bowler], ignore_index=True)
            time.sleep(0.1)
        else:
            print(f"Failed to retrieve data for player ID {p_id}, status code: {response.status_code}")

    filepath = Path('Csv/Yearly_stats_bowler.csv')
    yearly_stats_bowler_merged.to_csv(filepath, index_label="row_id")
    print(yearly_stats_bowler_merged)

#Main function below:

In [ ]:
#234675: Ravindrasinh Anirudhsinh Jadeja (ALL)
#625371: Hardik Himanshu Pandya (ALL)
#26421: Ravichandran Ashwin (ALL)
#625383: Jasprit Jasbirsingh Bumrah (BOW)
#481896: Mohammed Shami Ahmed (BOW)
#326016: Bhuvneshwar Kumar Singh (BOW)
#376116: Umeshkumar Tilak Yadav (BOW)

player_ids=['234675', '625371', '26421', '625383', '481896', '326016', '376116']

os.system("rm -f Csv/*_bowler.csv")

career_avg_bowler(player_ids)
vs_country_bowler(player_ids)
home_vs_away_bowler(player_ids)
yearly_stats_bowler(player_ids)
